In [ ]:
import numpy as np
import pandas as pd


def connect_string(x, ms):
    x = list(map(lambda i: sorted(i.split(ms)), x))
    l = len(x[0])
    r = []
    for i in range(len(x)):
        for j in range(i, len(x)):
            if x[i][:l - 1] == x[j][:l - 1] and x[i][l - 1] != x[j][l - 1]:
                r.append(x[i][:l - 1] + sorted([x[j][l - 1], x[i][l - 1]]))
    return r


def find_rule(d, support, confidence, ms=u'-'):
    # 存储输出结果
    result = pd.DataFrame(index=['support', 'confidence'])

    # 1项集的支持度序列
    support_series = 1.0 * d.sum(axis=0) / d.shape[0]
    # 基于给定的最小支持度进行筛选,得到1项频繁集
    column = list(support_series[support_series > support].index)

    # 当1项频繁集个数大于1时
    k = 0
    while len(column) > 1:
        k = k + 1
        print(u'\n正在进行第%s次搜索...' % k)
        column = connect_string(column, ms)
        print(u'数目：%s...' % len(column))
        # 乘积为1表示两个项集同时发生，乘积为0表示不同发生
        sf = lambda i: d[i].prod(axis=1, numeric_only=True)  # 新一批支持度的计算函数

        # 创建连接数据，这一步耗时、耗内存最严重。当数据集较大时，可以考虑并行运算优化。
        d_2 = pd.DataFrame(list(map(sf, column)), index=[ms.join(i) for i in column]).T

        # 计算连接后的支持度
        support_series_2 = 1.0 * d_2[[ms.join(i) for i in column]].sum() / len(d)
        column = list(support_series_2[support_series_2 > support].index)  # 新一轮支持度筛选
        support_series = support_series.append(support_series_2)

        column2 = []
        # 遍历可能的推理，如{A,B,C}究竟是A+B-->C还是B+C-->A还是C+A-->B？
        for i in column:
            i = i.split(ms)
            for j in range(len(i)):
                column2.append(i[:j] + i[j + 1:] + i[j:j + 1])

        # 定义置信度序列
        cofidence_series = pd.Series(index=[ms.join(i) for i in column2])
        # 计算置信度序列
        for i in column2:
            cofidence_series[ms.join(i)] = support_series[ms.join(sorted(i))] / support_series[ms.join(i[:len(i) - 1])]

        for i in cofidence_series[cofidence_series > confidence].index:  # 置信度筛选
            result[i] = 0.0
            result[i]['confidence'] = cofidence_series[i]
            result[i]['support'] = support_series[ms.join(sorted(i.split(ms)))]

    result = result.T.sort_values(['confidence', 'support'], ascending=False)  # 结果整理，输出
    print(u'\n结果为：')
    print(result)
    return result

import chardet

with open(r"C:\Users\25809\Downloads\mobile_services.csv", 'rb') as file:
    rawdata = file.read(10000)
    result = chardet.detect(rawdata)
    encoding = result['encoding']
    print(f"Detected encoding: {encoding}")

if __name__ == '__main__':
    # 加载数据
    data = pd.read_csv(r'C:\Users\25809\Downloads\mobile_services.csv', encoding=encoding)
    print('转换原数据到0-1矩阵')
    ct = lambda x: pd.Series(1, index=x[pd.notnull(x)])
    b = map(ct, data.values)
    data = pd.DataFrame(list(b)).fillna(0)
    # 删除中间变脸b
    del b

    support = 0.2  # 最小支持度
    confidence = 0.5  # 最小置信度

    find_rule(data, support, confidence)


In [ ]:
import numpy as np
import pandas as pd
import xlrd
from mlxtend.frequent_patterns import apriori  # 生成频繁项集
from mlxtend.frequent_patterns import association_rules  # 生成强关联规则
import warnings

warnings.filterwarnings("ignore")  # 用于排除警告

import chardet

# Read the first few bytes to detect encoding
with open(r"C:\Users\25809\Downloads\mobile_services.csv", 'rb') as file:
    rawdata = file.read(10000)
    result = chardet.detect(rawdata)
    encoding = result['encoding']
    print(f"Detected encoding: {encoding}")

def loaddata():
    order_data = pd.read_csv(r"C:\Users\25809\Downloads\mobile_services.csv", header=0, encoding=encoding)


if __name__ == '__main__':
    dataSet = loaddata()
    column_list = []
    for var in dataSet:
        column_list = set(column_list) | set(var)
    print('转换原数据到0-1矩阵')
    data = pd.DataFrame(np.zeros((len(dataSet), 169)), columns=column_list)
    for i in range(len(dataSet)):
        for j in dataSet[i]:
            data.loc[i, j] += 1
    # apriori算法
    frequent_itemsets = apriori(data, min_support=0.02, use_colnames=True)
    print(pd.DataFrame(frequent_itemsets))
    pd.DataFrame(frequent_itemsets).to_csv('frequent_itemsets.csv')
    # 生成关联准则
    rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.35)
    print(pd.DataFrame(rules))
    pd.DataFrame(rules).to_csv('rules.csv')
